Thinc is this new ML library that I think is a PyTorch/TF type-checked wrapper? Lets find out...

https://thinc.ai/docs/

In [2]:
!pip3 install "thinc==8.0.0a1";
# pip3 install thinc[blis,cuda100]==8.0.0a1

Defaulting to user installation because normal site-packages is not writeable


If you have a GPU you should install the thinc-cuda version.

In [4]:
from thinc.api import prefer_gpu
prefer_gpu()

False

## Type checking

This is probably the coolest Thinc feature. Does it natively support JN?

It makes sure that the ouptut matches the input of the next layer for example. Very helpful.

You can also define your own datatypes.

https://thinc.ai/static/fbdab0fcfb4cf1df092dea72df276cc3/f7e02/type_checking2.jpg

## Functional API like Keras.

In [10]:
from thinc.api import chain, Relu, Softmax
 
n_hidden = 32
dropout = 0.2

model = chain(
    Relu(nO=n_hidden, dropout=dropout), 
    Relu(nO=n_hidden, dropout=dropout), 
    Softmax()
)

## Operator Overloading

I think this is a cool Thinc feature. Does something similar exist for PyTorch?

In [7]:
from thinc.api import Model, chain, Relu, Softmax
 
n_hidden = 32
dropout = 0.2

with Model.define_operators({">>": chain}):
    model = Relu(nO=n_hidden, dropout=dropout) >> Relu(nO=n_hidden, dropout=dropout) >> Softmax()

## Config file stuff

**from docs:**

Thinc provides a config system that lets you easily describe arbitrary trees of objects. The objects can be created via function calls you register using a simple decorator syntax. The config can include values like hyperparameters or training settings (whatever you need), or references to functions and the values of their arguments. Thinc will then construct the config bottom-up – so you can define one function with its arguments, and then pass the return value into another function.

--> I like this config as code concept.

In [9]:
from thinc.api import Config, registry

EXAMPLE_CONFIG1 = """
[hyper_params]
learn_rate = 0.001

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""

config1 = Config().from_str(EXAMPLE_CONFIG1)
config1



{'hyper_params': {'learn_rate': 0.001},
 'optimizer': {'@optimizers': 'Adam.v1', 'learn_rate': 0.001}}

## You can wrap TensorFlow, PyTorch and MXNet models 

In [12]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from thinc.api import TensorFlowWrapper, Adam

width = 32
nO = 10
nI = 784
dropout = 0.2

tf_model = Sequential()
tf_model.add(Dense(width, activation="relu", input_shape=(nI,)))
tf_model.add(Dropout(dropout))
tf_model.add(Dense(width, activation="relu", input_shape=(nI,)))
tf_model.add(Dropout(dropout))
tf_model.add(Dense(nO, activation="softmax"))

wrapped_tf_model = TensorFlowWrapper(tf_model)
wrapped_tf_model